In [1]:
import re

def tokenize(text):
    text = re.sub(r"(\w+)-\s*\n\s*(\w+)", r"\1\2", text)
    words = re.findall(r"\b[а-яё]+(?:[-][а-яё]+)*\b", text, re.IGNORECASE)
    return words


def preprocess(text):
    text = text.lower()
    removed_html = re.sub(r"<.*?>", "", text)
    removed_url = re.sub(r"https?://\S+|www\.\S+", "", removed_html)
    return text


text = ""
with open(r"book.txt", "r", encoding="utf-8") as txt:
    text = preprocess(txt.read())

In [2]:
tokens = tokenize(text)

In [3]:
from collections import Counter
import pymorphy3

morph = pymorphy3.MorphAnalyzer()


def lemmatize_and_filter(words):
    lemmas = []
    for word in words:
        parsed = morph.parse(word)[0]
        lemma = parsed.normal_form
        lemmas.append((lemma, parsed.tag.POS))
    return lemmas


unigrams = lemmatize_and_filter(tokens)
bigrams = map(
    "_".join,
    [
        (first[0], second[0])
        for first, second in zip(unigrams, unigrams[1:])
        if first[1] == "ADJF"
        and second[1] == "NOUN"
        and first[0] not in {"сам", "тот", "каждый", "такой", "один"}
    ],
)
unigrams = [unigram for unigram, _ in unigrams]

In [4]:
counter_f_uni = Counter(unigrams)
counter_f_bi = Counter(bigrams)

In [5]:
phrases_freq = counter_f_bi.most_common(20)

In [6]:
import numpy as np


def count_by_mi(counter_f_bi, counter_f_uni):
    counter_by_mi = {}
    N = len(counter_f_uni.keys())
    for bi, freq in counter_f_bi.items():
        a, b = bi.split("_")
        counter_by_mi[bi] = np.log2(N * freq / (counter_f_uni[a] * counter_f_uni[b]))
    return Counter(counter_by_mi)


counter_by_mi = count_by_mi(counter_f_bi, counter_f_uni)
phrases_mi = counter_by_mi.most_common(20)

In [7]:
def count_by_mi3(counter_f_bi, counter_f_uni):
    counter_by_mi3 = {}
    N = len(counter_f_uni.keys())
    for bi, freq in counter_f_bi.items():
        a, b = bi.split("_")
        counter_by_mi3[bi] = np.log2(
            N * freq**3 / (counter_f_uni[a] * counter_f_uni[b])
        )
    return Counter(counter_by_mi3)


counter_by_mi3 = count_by_mi3(counter_f_bi, counter_f_uni)
phrases_mi3 = counter_by_mi3.most_common(20)

In [ ]:
phrases_freq # 11/20

[('оценочный_функция', 151),
 ('оптимальный_стратегия', 78),
 ('случайный_величина', 64),
 ('простой_итерация', 43),
 ('суррогатный_функция', 42),
 ('градиентный_спуск', 37),
 ('текущий_состояние', 36),
 ('текущий_стратегия', 35),
 ('будущий_награда', 34),
 ('правый_часть', 33),
 ('следующий_состояние', 31),
 ('очередной_шаг', 31),
 ('внутренний_мотивация', 30),
 ('следующий_образ', 30),
 ('детерминированный_стратегия', 30),
 ('эволюционный_стратегия', 29),
 ('нижний_оценка', 29),
 ('несмещённый_оценка', 28),
 ('терминальный_состояние', 27),
 ('следующий_шаг', 26)]

In [ ]:
phrases_mi # 4/20

[('поведенческий_психология', np.float64(12.05697633617822)),
 ('лапласовский_детерминизм', np.float64(12.05697633617822)),
 ('кожаный_мешок', np.float64(12.05697633617822)),
 ('императорский_пингвин', np.float64(12.05697633617822)),
 ('исторический_маркёр', np.float64(12.05697633617822)),
 ('инопланетный_корабль', np.float64(12.05697633617822)),
 ('юбилейный_покупатель', np.float64(12.05697633617822)),
 ('субъективный_незнание', np.float64(12.05697633617822)),
 ('линейно-квадратичный_регулятор', np.float64(12.05697633617822)),
 ('вражеский_нло', np.float64(11.05697633617822)),
 ('автономный_автомобиль', np.float64(11.05697633617822)),
 ('англоязычный_литература', np.float64(11.05697633617822)),
 ('математический_анализ', np.float64(11.05697633617822)),
 ('красивый_символ', np.float64(11.05697633617822)),
 ('лесной_тропинка', np.float64(11.05697633617822)),
 ('геометрический_прогрессия', np.float64(10.472013835457062)),
 ('местный_терминология', np.float64(10.472013835457062)),
 ('весё

In [ ]:
phrases_mi3 # 16/20

[('оценочный_функция', np.float64(16.66149315605713)),
 ('экспоненциальный_сглаживание', np.float64(16.308515103174184)),
 ('градиентный_спуск', np.float64(16.286058508556103)),
 ('случайный_величина', np.float64(16.272443726034474)),
 ('временной_разность', np.float64(16.25093502982019)),
 ('марковский_цепь', np.float64(16.176451092516928)),
 ('динамический_программирование', np.float64(16.14889882561926)),
 ('шумный_телевизор', np.float64(15.621490000873074)),
 ('внутренний_мотивация', np.float64(15.492245904141527)),
 ('правый_часть', np.float64(15.488852232294954)),
 ('крайний_мера', np.float64(15.163942053742637)),
 ('простой_итерация', np.float64(15.12297026585508)),
 ('нормировочный_константа', np.float64(14.867606390268683)),
 ('многорукий_бандит', np.float64(14.6505008504028)),
 ('репараметризационный_трюк', np.float64(14.641938836899376)),
 ('краткий_маршрут', np.float64(14.565123239848544)),
 ('локальный_оптимум', np.float64(14.319169765572642)),
 ('центральный_сервер', np.f

# Отчёт

В ходе выполнения лабораторной работы были реализованы следующие методы 
нахождения словосочетаний: простой частотный, мера взаимной информации 
и мера взаимной информации с кубом.  

Результаты в целом соответствуют ожиданиям. Простой частотный метод выявил 
в основном обычные пары прилагательного и существительного 
(`текущий_состояние`, `правый_часть`), 
однако также встретились реальные термины, например 
`оценочный_функция` или `детерминированный_стратегия`. 
Метод, основанный на мере взаимной информации, тоже нашел несколько терминов 
(`линейно-квадратичный_регулятор`, `геометрический_прогрессия`), 
но обнаружил довольно также довольно странные и очень редкие сочетания, 
например `местный_терминология` или `императорский_пингвин`
 встречались в книге явно один раз.

Наиболее качественные результаты продемонстрировал метод, использующий 
меру взаимной информации с кубом. С его помощью было выявлено значительное 
количество терминов, включая `экспоненциальный_сглаживание`, `марковский_цепь`, 
`многорукий_бандит` или `простой_итерация`.

## Точность (по терминам)

* Частотный метод `11/20`
* Взаимная информация `4/20`
* Взаимная информация с кубом `16/20`